<a href="https://colab.research.google.com/github/paulo-marquesmorgado/EConcepts/blob/main/Assignm1_2024-25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Economic Concepts Week 3, Assignment 2

## Macro Economic Analysis

The objective of this assignment is to guide you through a reliable and rich source of macroeconomic data and to
show how much you can learn from it by simply plotting the data and making some straightforward calculations.


In [1]:
StudentName   = "Klein"
StudentNumber = "1234567"
WEB_Data_Github_location = 'https://github.com/paulo-marquesmorgado/EConcepts/blob/main/SNA_TABLE1_ARCHIVE_ALL-1950-1918.csv'
# Raw URL for the CSV file on GitHub
file_name = "SNA_TABLE1_ARCHIVE_ALL-1950-1918.csv"
Data_Github_location = 'https://raw.githubusercontent.com/paulo-marquesmorgado/EConcepts/main/SNA_TABLE1_ARCHIVE_ALL-1950-1918.csv'
assignment="assignment_2"
myheader = 'Economic Concepts Assignment 1 Report'
mytitle = 'Inventory Investment Analysis'
mysource = 'OECD statistics at stats.oecd.org'

In [2]:
!pip install fpdf

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from fpdf import FPDF
import os

## Support Functions to Calculate the statistics

In [4]:
def calculate_and_display_stats(this_df):
    # Compute statistics
    max_inventory_investment = this_df['Inv_Perc_GDP'].max()
    min_inventory_investment = this_df['Inv_Perc_GDP'].min()
    mean_inventory_investment = this_df['Inv_Perc_GDP'].mean()
    std_dev_inventory_investment = this_df['Inv_Perc_GDP'].std()

    # Display the results
    print(f"Max Inventory Investment as Percentage of GDP: {max_inventory_investment:.2f}")
    print(f"Min Inventory Investment as Percentage of GDP: {min_inventory_investment:.2f}")
    print(f"Standard Deviation of Inventory Investment as Percentage of GDP: {std_dev_inventory_investment:.2f}")
    print("")
    print(f"It's important to consider the context of each year to interpret these values effectively.")
    print(f"For instance, negative percentages may indicate destocking or economic challenges,")
    print(f"while positive percentages could imply inventory buildup or economic expansion.")


### Create the function process_dataframe(df)  
The provided code snippet serves to manipulate a DataFrame called 'df' with the goal of creating a new DataFrame 'df_new' that retains specific columns, compacts the DataFrame's index, and removes duplicate rows based on a particular column.

Then reshape and prepare a DataFrame named 'df_pivot' for analysis by pivoting df, renaming columns, and resetting the index.
Finally, perform data operations on the DataFrames 'df_new' and 'df_pivot,' with the ultimate goal of merging them based on the 'Year' column and calculating the inventory investment as a percentage of GDP.

In [5]:
def process_dataframe(df):
    # Create a new DataFrame 'df_new'
    selected_columns = ['LOCATION', 'Country', 'Year']
    df_new = df[selected_columns]
    # Reset the index to compact the DataFrame
    df_new.reset_index(drop=True, inplace=True)
    # Drop duplicates based on 'Year'
    df_new = df_new.drop_duplicates(subset='Year', keep='first')

    # Pivot the DataFrame to separate "Inventory" and "GDP" columns
    df_pivot = df.pivot(index='Year', columns='Transaction', values='Value')
    # Rename the columns to match your desired column names
    # df_pivot = df_pivot.rename(columns={'Changes in inventories': 'Inventory', 'Gross domestic product (expenditure approach)': 'GDP'})
    # Reset the index to make "Year" a regular column
    df_pivot.reset_index(inplace=True)

    # Merge the DataFrames df_new and df_pivot on 'Year'
    merged_df = df_new.merge(df_pivot, on=['Year'])
    # Compute inventory investment as a percentage of GDP
    # merged_df['Inv_Perc_GDP'] = (merged_df['Inventory'] / merged_df['GDP']) * 100
    merged_df['Inv_Perc_GDP'] = (merged_df['Changes in inventories'] / merged_df['Gross domestic product (expenditure approach)']) * 100
    merged_df = merged_df.dropna(subset=['Inv_Perc_GDP'])
    return merged_df



---

# STEP 1: Load the CSV file

In [6]:
# Step 1: Load the CSV file
original_df = pd.read_csv(Data_Github_location)
# print(original_df.head())

# Create the 'LOCATION_Year' column by concatenating 'LOCATION' and 'Year'
original_df['LOCATION_Year'] = original_df['LOCATION'].astype(str) + '_' + original_df['Year'].astype(str)
# print(original_df)
# original_df['Transaction'].unique()

# STEP 2: Display Graph and Calculations

In [7]:
import os
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns

# Create a dropdown widget to select a country
country_dropdown = widgets.Dropdown(options=original_df['Country'].unique(), description='Country:')
# Create widgets to select start_year and end_year
start_year_slider = widgets.IntSlider(min=1970, max=2017, step=1, value=1990, description='Start Year:')
end_year_slider = widgets.IntSlider(min=1970, max=2017, step=1, value=2017, description='End Year:')

# Define a function to update the plot based on the selected country and year range
def update_plot(selected_country, start_year, end_year):
    # Filter the DataFrame based on the selected country and year range
    df = original_df[
        (original_df['Transaction'].isin(['Changes in inventories', 'Gross domestic product (expenditure approach)'])) &
        (original_df['Country'] == selected_country) &
        (original_df['Year'] >= start_year) &
        (original_df['Year'] <= end_year)
    ]

    # Process the filtered DataFrame
    merged_df = process_dataframe(df)

    # Plot the graph
    plt.figure(figsize=(10, 6))
    sns.lineplot(data=merged_df, x='Year', y='Inv_Perc_GDP', marker='o')
    title = f'Inventory Investment as Percentage of GDP Over Time - {selected_country}'
    plt.title(title)
    plt.xlabel('Year')
    plt.ylabel('Inventory Investment as Percentage of GDP')
    plt.grid(True)
    plt.show()
    # Call the statistics with your 'merged_df' DataFrame
    calculate_and_display_stats(merged_df)

# Define an observer to update the plot when the dropdown value changes
widgets.interactive(update_plot, selected_country=country_dropdown, start_year=start_year_slider, end_year=end_year_slider)


interactive(children=(Dropdown(description='Country:', options=('Australia', 'Austria', 'Belgium', 'Canada', '…

# Other Preps

In [16]:
!pip install vega

In [17]:
# Import necessary libraries
import os
import pandas as pd
import ipywidgets as widgets
import altair as alt  # Altair for Vega-Lite plotting
from IPython.display import display
import seaborn as sns

# Create a dropdown widget to select a country
country_dropdown = widgets.Dropdown(options=original_df['Country'].unique(), description='Country:')

# Create widgets to select start_year and end_year
start_year_slider = widgets.IntSlider(min=1970, max=2017, step=1, value=1990, description='Start Year:')
end_year_slider = widgets.IntSlider(min=1970, max=2017, step=1, value=2017, description='End Year:')

# Define a function to update the plot based on the selected country and year range
def update_plot(selected_country, start_year, end_year):
    # Filter the DataFrame based on the selected country and year range
    df = original_df[
        # (original_df['Transaction'] == selected_transaction) &
        (original_df['Transaction'].isin(['Changes in inventories', 'Gross domestic product (expenditure approach)'])) &
        (original_df['Country'] == selected_country) &
        (original_df['Year'] >= start_year) &
        (original_df['Year'] <= end_year)
    ]

    # Process the filtered DataFrame
    merged_df = process_dataframe(df)

    # Create the Vega-Lite chart
    chart = alt.Chart(merged_df).mark_line(point=True).encode(
        x='Year:O',  # Year as ordinal (categorical)
        y='Inv_Perc_GDP:Q',  # Inv_Perc_GDP as quantitative
        color=alt.value('blue')  # Set color to blue
    ).properties(
        width=800,  # Set chart width
        height=400,  # Set chart height
        title=f'Inventory Investment as Percentage of GDP Over Time - {selected_country}'
        # title=f'{selected_country} ({selected_transaction})'
    )

    # Show the chart
    display(chart)

    # Call the statistics with your 'merged_df' DataFrame
    calculate_and_display_stats(merged_df)

# Define an observer to update the plot when the dropdown value changes
widgets.interactive(update_plot, selected_country=country_dropdown, start_year=start_year_slider, end_year=end_year_slider)



interactive(children=(Dropdown(description='Country:', options=('Australia', 'Austria', 'Belgium', 'Canada', '…

In [19]:
# Import the necessary libraries:
import os
import pandas as pd
import ipywidgets as widgets
import altair as alt
from IPython.display import display
import seaborn as sns

# Create a dropdown widget to select a country:
country_dropdown = widgets.Dropdown(options=original_df['Country'].unique(), description='Country:')

# Create a dropdown widget to select a transaction:
# transaction_dropdown = widgets.Dropdown(options=original_df['Transaction'].unique(), description='Transaction:')
transaction_dropdown = widgets.Dropdown(options=original_df['Transaction'].unique(), description='Transaction:', style={'description_width': 'initial', 'width': '500px'})

# Create widgets to select start_year and end_year:
start_year_slider = widgets.IntSlider(min=1970, max=2017, step=1, value=1990, description='Start Year:')
end_year_slider = widgets.IntSlider(min=1970, max=2017, step=1, value=2017, description='End Year:')

# Modify the update_plot function to include the selected_transaction argument and filter the DataFrame based on the selected country, transaction, and year range:
def update_plot(selected_country, selected_transaction, start_year, end_year):
    # Filter the DataFrame based on the selected country, transaction, and year range
    df = original_df[
        (original_df['Transaction'] == selected_transaction) &
        (original_df['Country'] == selected_country) &
        (original_df['Year'] >= start_year) &
        (original_df['Year'] <= end_year)
    ]

    # Process the filtered DataFrame
    merged_df = process_dataframe(df)

    # Create the Altair chart
    chart = alt.Chart(merged_df).mark_line(point=True).encode(
        x='Year:O',  # Year as ordinal (categorical)
        # y='Inv_Perc_GDP:Q',  # Inv_Perc_GDP as quantitative
        y=alt.value(selected_transaction),  # selected transaction as quantitative
        color=alt.value('blue')  # Set color to blue
    ).properties(
        width=800,  # Set chart width
        height=400,  # Set chart height
        title=f'{selected_country} ({selected_transaction})'
        # title=f'Inventory Investment as Percentage of GDP Over Time - {selected_country} ({selected_transaction})'
    )

    # Show the chart
    display(chart)

    # Call the statistics with your 'merged_df' DataFrame
    calculate_and_display_stats(merged_df)

# Define an observer to update the plot when the dropdown value changes:
widgets.interactive(update_plot, selected_country=country_dropdown, selected_transaction=transaction_dropdown, start_year=start_year_slider, end_year=end_year_slider)


interactive(children=(Dropdown(description='Country:', options=('Australia', 'Austria', 'Belgium', 'Canada', '…

In [11]:
#Perp
import os
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns

def update_plot(selected_country, start_year, end_year):
    # Filter the DataFrame based on the selected country and year range
    df = original_df[
        (original_df['Transaction'].isin(['Changes in inventories', 'Gross domestic product (expenditure approach)'])) &
        (original_df['Country'] == selected_country) &
        (original_df['Year'] >= start_year) &
        (original_df['Year'] <= end_year)
    ]

    # Process the filtered DataFrame
    merged_df = process_dataframe(df)

    # Plot the graph
    plt.figure(figsize=(10, 6))
    sns.lineplot(data=merged_df, x='Year', y='Inv_Perc_GDP', marker='o')
    title = f'Inventory Investment as Percentage of GDP Over Time - {selected_country}'
    plt.title(title)
    plt.xlabel('Year')
    plt.ylabel('Inventory Investment as Percentage of GDP')
    plt.grid(True)
    plt.show()
    # Call the statistics with your 'merged_df' DataFrame
    calculate_and_display_stats(merged_df)

widgets.interactive(update_plot, selected_country=country_dropdown, start_year=start_year_slider, end_year=end_year_slider)


interactive(children=(Dropdown(description='Country:', options=('Australia', 'Austria', 'Belgium', 'Canada', '…

In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display, clear_output
from ipywidgets import Layout, Box

# Create a dropdown widget to select start_year and end_year
start_year_slider = widgets.IntSlider(min=1970, max=2017, step=1, value=1990, description='Start Year:')
end_year_slider = widgets.IntSlider(min=1970, max=2017, step=1, value=2017, description='End Year:')

# Create a multi-choice widget to select countries
countries_options = sorted(original_df['Country'].unique())
default_country = 'Germany' if 'Germany' in countries_options else countries_options[0]  # Set default to 'Germany' if available, else use the first option

# Create a multi-choice widget to select countries
selected_countries_selector = widgets.SelectMultiple(
    options=sorted(original_df['Country'].unique()),  # Use merged_df_base instead of original_df
    value=[default_country],  # Default selected country
    description='Select Countries: ',
    rows=10  # Adjust the number of visible rows as needed
)

# Get a list of unique transactions from the columns of original_df
# unique_transactions = [col for col in original_df.columns if col not in ['LOCATION_Year', 'LOCATION', 'Country', 'Year']]
unique_transactions = original_df['Transaction'].unique() #, description='Transaction:')#, style={'description_width': 'initial', 'width': '500px'})
print(unique_transactions[0])

# Create a multi-choice widget to select transactions
selected_transactions_selector = widgets.SelectMultiple(
    options=unique_transactions,
    value=[unique_transactions[0]],
    description='Select Transactions:',
    rows=10
)

# Create a plot output area
plot_output = widgets.Output()

# Define a function to update the plot based on the selected countries, transactions, and year range
def update_plot(selected_countries, selected_transactions, start_year, end_year):
    with plot_output:
        clear_output(wait=True)  # Clear previous plot
        # Filter the DataFrame based on the selected countries, transactions, and year range
        # Process the filtered DataFrame
        df_new = original_df[
            (original_df['Country'].isin(selected_countries)) &
            (original_df['Transaction'].isin(selected_transactions)) &
            (original_df['Year'] >= start_year) &
            (original_df['Year'] <= end_year)
        ]
        merged_df = process_dataframe(df_new)
        print(merged_df)

        # # Process the filtered DataFrame
        # for transaction in selected_transactions:
        #   plt.figure(figsize=(8, 6))
        #   plt.title(f'{transaction} per Year')
        #   plt.xlabel('Year')
        #   plt.ylabel(transaction)

        #   for thiscountry in selected_countries:
        #     print(thiscountry)
        #     country_df = df_new[(df_new['Country'] == thiscountry) & (df_new['Transaction'] == transaction)]
        #     plt.plot(country_df['Year'], country_df[transaction], marker='o', label=f'{thiscountry} - {transaction}')
        #   plt.legend(title='Countries')
        #   plt.grid(True)  # Add grid lines to the plot
        #   plt.show()

# Create an observer to update the plot when the selection or year range changes
controls = widgets.VBox([start_year_slider, end_year_slider, selected_countries_selector, selected_transactions_selector])
box_layout = Layout(display='flex', flex_flow='row wrap', justify_content='space-between', width='100%')
app_layout = Box(children=[controls, plot_output], layout=box_layout)
widgets.interactive(update_plot, selected_countries=selected_countries_selector,
                    selected_transactions=selected_transactions_selector,
                    start_year=start_year_slider, end_year=end_year_slider)

display(app_layout)


Gross domestic product (output approach)


Box(children=(VBox(children=(IntSlider(value=1990, description='Start Year:', max=2017, min=1970), IntSlider(v…

In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display, clear_output
from ipywidgets import Layout, Box

# Create a dropdown widget to select start_year and end_year
start_year_slider = widgets.IntSlider(min=1970, max=2017, step=1, value=1990, description='Start Year:')
end_year_slider = widgets.IntSlider(min=1970, max=2017, step=1, value=2017, description='End Year:')

# Create a multi-choice widget to select countries
countries_options = sorted(original_df['Country'].unique())
default_country = 'Germany' if 'Germany' in countries_options else countries_options[0]  # Set default to 'Germany' if available, else use the first option

# Create a multi-choice widget to select countries
selected_countries_selector = widgets.SelectMultiple(
    options=sorted(original_df['Country'].unique()),  # Use merged_df_base instead of original_df
    value=[default_country],  # Default selected country
    description='Select Countries: ',
    rows=10  # Adjust the number of visible rows as needed
)

# Get a list of unique transactions from the columns of original_df
# unique_transactions = [col for col in original_df.columns if col not in ['LOCATION_Year', 'LOCATION', 'Country', 'Year']]
unique_transactions = original_df['Transaction'].unique() #, description='Transaction:')#, style={'description_width': 'initial', 'width': '500px'})
print(unique_transactions[0])

# Create a multi-choice widget to select transactions
selected_transactions_selector = widgets.SelectMultiple(
    options=unique_transactions,
    value=[unique_transactions[0]],
    description='Select Transactions:',
    rows=10
)

# Create a plot output area
plot_output = widgets.Output()

# Define a function to update the plot based on the selected countries, transactions, and year range
def update_plot(selected_countries, selected_transactions, start_year, end_year):
    with plot_output:
        clear_output(wait=True)  # Clear previous plot

        for transaction in selected_transactions:
            plt.figure(figsize=(8, 6))
            plt.title(f'{transaction} per Year')
            plt.xlabel('Year')
            plt.ylabel(transaction)

            for thiscountry in selected_countries:
                print(thiscountry)
                country_df = original_df[
                    (original_df['Country'] == thiscountry) &
                    (original_df['Transaction'] == transaction) &
                    (original_df['Year'] >= start_year) &
                    (original_df['Year'] <= end_year)
                ]

                plt.plot(country_df['Year'], country_df[transaction], marker='o', label=f'{thiscountry} - {transaction}')

            plt.legend(title='Countries')
            plt.grid(True)  # Add grid lines to the plot
            plt.show()


# Create an observer to update the plot when the selection or year range changes
controls = widgets.VBox([start_year_slider, end_year_slider, selected_countries_selector, selected_transactions_selector])
box_layout = Layout(display='flex', flex_flow='row wrap', justify_content='space-between', width='100%')
app_layout = Box(children=[controls, plot_output], layout=box_layout)
widgets.interactive(update_plot, selected_countries=selected_countries_selector,
                    selected_transactions=selected_transactions_selector,
                    start_year=start_year_slider, end_year=end_year_slider)

display(app_layout)


Gross domestic product (output approach)


Box(children=(VBox(children=(IntSlider(value=1990, description='Start Year:', max=2017, min=1970), IntSlider(v…